In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.rcParams['figure.figsize'] = 10, 8

# Sodium intake and blood pressure
The following example is taken from Introduction to Causal Inference from a Machine Learning Perspective by Brady Neal (https://www.bradyneal.com/Introduction_to_Causal_Inference-Dec9_2020-Neal.pdf), who adapted it from Luque-Fernandez et al. (2018), 'Educational Note: Paradoxical collider effect in the analysis of non-communicable disease epidemiological data: a reproducible illustration and web application'.

The outcome of interest is (systolic) blood pressure. This is an important outcome because a large proportion of adults have high blood pressure, and high blood pressure is associated with increased risk of mortality. The “treatment” of interest is sodium intake. Sodium intake is a continuous variable; we will binarize it by letting $T = 1$ denote daily sodium intake above 3.5 grams and letting $T = 0$ denote daily sodium intake below 3.5 grams. We will be estimating the causal effect of sodium intake on blood pressure. In our data, we also have the age
of the individuals and amount of protein in their urine (proteinuria) as covariates.

Luque-Fernandez et al. run a simulation, taking care to be sure that the range of values is “biologically plausible and as close to reality as possible.”

In [2]:
def generate_data(n=1000, seed=0, beta1=1.05, alpha1=0.4, alpha2=0.3, binary_treatment=True, binary_cutoff=3.5):
    np.random.seed(seed)
    age = np.random.normal(65, 5, n)
    sodium = age / 18 + np.random.normal(size=n)
    if binary_treatment:
        if binary_cutoff is None:
            binary_cutoff = sodium.mean()
        sodium = (sodium > binary_cutoff).astype(int)
    blood_pressure = beta1 * sodium + 2 * age + np.random.normal(size=n)
    proteinuria = alpha1 * sodium + alpha2 * blood_pressure + np.random.normal(size=n)
    return pd.DataFrame({'blood_pressure': blood_pressure, 'sodium': sodium,
                         'age': age, 'proteinuria': proteinuria})

df = generate_data(beta1=1.05, alpha1=.4, alpha2=.3, binary_treatment=True, n=10000)
df.describe()

,blood_pressure,sodium,age,proteinuria
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,130.380570,0.543400,64.907831,39.329525
std,10.049835,0.498138,4.938030,3.238240
min,93.920465,0.000000,46.299497,27.350540
25%,123.526001,0.000000,61.551466,37.090305
50%,130.323792,1.000000,64.864637,39.308601
75%,137.087790,1.000000,68.230445,41.491794
max,169.056247,1.000000,84.008301,51.247123


Because we simulate the data, we know that the true ATE of sodium on blood pressure is 1.05. Let's try to estimate it to see how closely the estimate would match the true value.

Naive estimator of ATE:

In [3]:
df.blood_pressure[df.sodium == 1].mean() - df.blood_pressure[df.sodium == 0].mean()

5.179692033288873

As you can see, it's very biased because the data are not from a randomized experiment – there are features we need to adjust for.

# Regression

Let's try taking two covariates into account, and use linear regression to estimate ATE:

In [4]:
m = smf.ols('blood_pressure ~ sodium + age + proteinuria', data=df)
fitted = m.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:         blood_pressure   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 3.661e+05
Date:                Mon, 18 Jan 2021   Prob (F-statistic):               0.00
Time:                        01:10:53   Log-Likelihood:                -13722.
No. Observations:               10000   AIC:                         2.745e+04
Df Residuals:                    9996   BIC:                         2.748e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0307      0.127     -0.242      

ATE estimate is the coeficient if sodium variable - 0.84. Much better, but still not very close. Given that the dataset is quite large (and we could make it as large as we want, and the estimate would still be not very close to the true value), it indicates that there's bias. 

Here's where it comes from:

![title](sodium_DAG.png)

Proteinuria variable here is a collider, so adjusting for it introduces additional bias. We should have drawn the DAG before trying to estimate ATE!

With the DAG of the problem it's clear now that we only have to adjust for age. Let's see what we'd get:

In [5]:
m = smf.ols('blood_pressure ~ sodium + age', data=df)
fitted = m.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:         blood_pressure   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 5.026e+05
Date:                Mon, 18 Jan 2021   Prob (F-statistic):               0.00
Time:                        01:10:53   Log-Likelihood:                -14162.
No. Observations:               10000   AIC:                         2.833e+04
Df Residuals:                    9997   BIC:                         2.835e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0265      0.133     -0.200      0.8

Very close to the true effect now! Note that the coefficient is significantly different from 0 (P>|t| column), and the 95% confidence interval for it contains true value.

# Propensity score weighting

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

# classifier to estimate the propensity score
cls = LogisticRegression()

# calibration of the classifier
cls = CalibratedClassifierCV(cls)

X = df[['age']]
y = df['sodium']
cls.fit(X, y)
df['e'] = cls.predict_proba(X)[:,1].tolist()
df.head()

,blood_pressure,sodium,age,proteinuria,e
0,149.020569,1,73.820262,45.477403,0.724639
1,134.001092,0,67.000786,40.505112,0.590681
2,141.655496,1,69.893690,43.400773,0.650624
3,153.887146,1,76.204466,46.701443,0.764467
4,147.221633,1,74.337790,45.220249,0.733633


In [7]:
df['w'] = df['sodium'] / df['e'] + (1 - df['sodium']) / (1 - df['e'])

Notice we calibrate the classifier above to have propensity scores accurately estimating the probabilities!

A simple way to estimate ATE is to calculate the difference of weighted means between treated and control groups. A slightly more smart way would be to pass these weights to the regression function – this way we'd also get p-value and confidence interval automatically. Notice we're using wls (weighted least squares) instead of the ols (ordinary least squares).

In [8]:
m = smf.wls('blood_pressure ~ sodium + age', data=df, weights=df['w'])
fitted = m.fit()
print(fitted.summary())

                            WLS Regression Results                            
Dep. Variable:         blood_pressure   R-squared:                       0.990
Model:                            WLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 4.953e+05
Date:                Mon, 18 Jan 2021   Prob (F-statistic):               0.00
Time:                        01:10:53   Log-Likelihood:                -14265.
No. Observations:               10000   AIC:                         2.854e+04
Df Residuals:                    9997   BIC:                         2.856e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0606      0.131     -0.461      0.6

# Doubly robust estimator

To build doubly robust estimator, we'd need two additional linear regression models, each using features from the selected adjustment set (which contains only age in this case and not proteinuria).

In [9]:
from sklearn.linear_model import LinearRegression

y0 = LinearRegression().fit(df[df.sodium == 0][['age']], df[df.sodium == 0]['blood_pressure']).predict(df[['age']])
y1 = LinearRegression().fit(df[df.sodium == 1][['age']], df[df.sodium == 1]['blood_pressure']).predict(df[['age']])

df['DR0'] = (1-df['sodium']) * (df['blood_pressure'] - y0)/(1-df['e']) + y0
df['DR1'] =    df['sodium']  * (df['blood_pressure'] - y1)/   df['e']  + y1

In [10]:
df['DR1'].mean() - df['DR0'].mean()

1.0578232752447718

# Matching with Machalanobis distance

For matching, we're going to use CausalInference library: https://github.com/laurencium/causalinference You'd have to install it, for example, using 'pip install CausalInference'. It's not very popular, so I'd be cautious of the results, but it's what we have.

In [11]:
from causalinference import CausalModel
adjustment_set = ['age']

causal = CausalModel(
    Y=df['blood_pressure'].values, # outcome
    D=df['sodium'].values, # treatment
    X=df[adjustment_set].values
)

In [12]:
causal.est_via_matching(bias_adj=True)
print(causal.estimates)

/Users/maratstelman/opt/anaconda3/lib/python3.8/site-packages/causalinference/estimators/matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef



Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      1.060      0.035     30.672      0.000      0.992      1.127
           ATC      1.053      0.040     26.533      0.000      0.975      1.131
           ATT      1.065      0.040     26.950      0.000      0.987      1.142



# Propensity score matching
We'll use the same library as above, but pass propensity score column for matching:

In [13]:
causal = CausalModel(
    Y=df['blood_pressure'].values, # outcome
    D=df['sodium'].values, # treatment
    X=df['e'].values
)
causal.est_via_matching(bias_adj=True)
print(causal.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      1.060      0.035     30.670      0.000      0.992      1.127
           ATC      1.054      0.040     26.538      0.000      0.976      1.131
           ATT      1.065      0.040     26.944      0.000      0.987      1.142

